## Import Libs 

In [1]:
#import basic Libraries
import pandas as pd
import numpy as np
import numpy_indexed as npi
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#combine sparse matrices
import scipy.sparse as sps

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import MiniBatchDictionaryLearning

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Webscraping (for locations)
#from bs4 import BeautifulSoup 
#import requests

#from z3c.batching.batch import Batch

from collections import defaultdict

%matplotlib inline

In [2]:
filenames = ['aisles','departments' ,'order_products__prior','order_products__train', 'orders','products']

In [3]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/'+ filenames[0] + '.csv')
departments = pd.read_csv('./data/'+ filenames[1] + '.csv')
order_products__prior = pd.read_csv('./data/'+ filenames[2] + '.csv')
order_products__train = pd.read_csv('./data/'+ filenames[3] + '.csv')
orders = pd.read_csv('./data/'+ filenames[4] + '.csv')
products = pd.read_csv('./data/'+ filenames[5] + '.csv')

## EDA

In [4]:
#checking the shape of each of the file and checks for any null values.
for name in filenames:
    print(name, eval(name).shape,": Null values?", eval(name).isnull().any()[0])

aisles (134, 2) : Null values? False
departments (21, 2) : Null values? False
order_products__prior (32434489, 4) : Null values? False
order_products__train (1384617, 4) : Null values? False
orders (3421083, 7) : Null values? False
products (49688, 4) : Null values? False


In [5]:
aisles.head()
departments.head()
#combine aisle and departments
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
products2 = pd.read_csv('./data/'+ filenames[5] + '.csv')
opt2 = order_products__train.merge(products2,how='left', on='product_id')
opt2 = opt2.merge(departments,how='left', on='department_id')
opt2 = opt2.merge(aisles,how='left', on='aisle_id')

In [11]:
# cheese blintzes is 4279 mebe
opt2[opt2['product_name'].str.contains('Cheese Blintzes')]['product_id'].unique()
# opt2[opt2['product_id']==2755]

array([4279], dtype=int64)

In [12]:
opt = order_products__train.merge(products,how='left', on='product_id')
opt = opt.merge(departments,how='left', on='department_id')
opt = opt.merge(aisles,how='left', on='aisle_id')

In [13]:
opt['aisle_id']= opt['aisle_id'].astype(str)

In [14]:
opt[['aisle','aisle_id']]['aisle'].value_counts().head()

fresh vegetables              150609
fresh fruits                  150473
packaged vegetables fruits     78493
yogurt                         55240
packaged cheese                41699
Name: aisle, dtype: int64

 ## Preparing for Market Basket Analysis##

In [136]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [137]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

# of unique aisles = 134


In [138]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [139]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [140]:
opt[opt['aisle_id'] == 134]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle


In [141]:
range(len(opt['aisle_id'].unique()))

range(0, 134)

In [142]:
opt[opt['aisle_id'] == '1']['aisle'].value_counts()

prepared soups salads    2936
Name: aisle, dtype: int64

In [143]:
#basket.drop(columns=0,inplace=True)
#basket.rename(columns=range(len(opt['aisle_id'].unique())).tolist())
#basket[0].sum()
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [145]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [146]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))
#aisle_dict = dict(zip((opt['aisle_id'].unique()),opt['aisle'].unique()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  if __name__ == '__main__':


In [147]:
aisle_dict
#len(basket_sets.columns)

{1: 'prepared soups salads',
 2: 'specialty cheeses',
 3: 'energy granola bars',
 4: 'instant foods',
 5: 'marinades meat preparation',
 6: 'other',
 7: 'packaged meat',
 8: 'bakery desserts',
 9: 'pasta sauce',
 10: 'kitchen supplies',
 11: 'cold flu allergy',
 12: 'fresh pasta',
 13: 'prepared meals',
 14: 'tofu meat alternatives',
 15: 'packaged seafood',
 16: 'fresh herbs',
 17: 'baking ingredients',
 18: 'bulk dried fruits vegetables',
 19: 'oils vinegars',
 20: 'oral hygiene',
 21: 'packaged cheese',
 22: 'hair care',
 23: 'popcorn jerky',
 24: 'fresh fruits',
 25: 'soap',
 26: 'coffee',
 27: 'beers coolers',
 28: 'red wines',
 29: 'honeys syrups nectars',
 30: 'latino foods',
 31: 'refrigerated',
 32: 'packaged produce',
 33: 'kosher foods',
 34: 'frozen meat seafood',
 35: 'poultry counter',
 36: 'butter',
 37: 'ice cream ice',
 38: 'frozen meals',
 39: 'seafood counter',
 40: 'dog food care',
 41: 'cat food care',
 42: 'frozen vegan vegetarian',
 43: 'buns rolls',
 44: 'eye ea

In [148]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [149]:
basket_sets.head()

,prepared soups salads,specialty cheeses,energy granola bars,instant foods,marinades meat preparation,other,packaged meat,bakery desserts,pasta sauce,kitchen supplies,...,trail mix snack mix,feminine care,body lotions soap,tortillas flat bread,frozen appetizers sides,hot cereal pancake mixes,dry pasta,beauty,muscles joints pain relief,specialty wines champagnes
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [150]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

## Market Basket Results

In [151]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [152]:
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(fresh fruits),(lunch meat),0.550099,0.108590,0.075521,0.137286,1.264255,0.015785,1.033262
1,(lunch meat),(fresh fruits),0.108590,0.550099,0.075521,0.695466,1.264255,0.015785,1.477341
2,(fresh fruits),(fresh dips tapenades),0.550099,0.102386,0.072312,0.131453,1.283891,0.015989,1.033466
3,(fresh dips tapenades),(fresh fruits),0.102386,0.550099,0.072312,0.706268,1.283891,0.015989,1.531668
4,(fresh fruits),(fresh vegetables),0.550099,0.450975,0.327333,0.595043,1.319458,0.079252,1.355760


In the above example, the {Diaper} is the **antecedent** and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [153]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,"(fresh fruits, packaged cheese)",(packaged vegetables fruits),0.160911,0.383030,0.098888,0.614550,1.604444,0.037254,1.600649
11,(packaged vegetables fruits),"(fresh fruits, packaged cheese)",0.383030,0.160911,0.098888,0.258173,1.604444,0.037254,1.131111
18,"(fresh fruits, packaged cheese, packaged veget...",(fresh vegetables),0.098888,0.450975,0.076626,0.774875,1.718220,0.032030,2.438753
19,"(fresh fruits, packaged cheese, fresh vegetables)",(packaged vegetables fruits),0.112538,0.383030,0.076626,0.680889,1.777637,0.033520,1.933399
20,"(fresh fruits, packaged vegetables fruits, fre...",(packaged cheese),0.203919,0.237781,0.076626,0.375766,1.580304,0.028138,1.221047


## Creating Recommender system

In [154]:
def mean_center_rows(df):
    return (df.T - df.mean(axis=1))

basket_mc = mean_center_rows(basket_sets)

In [155]:
basket_mc.shape

(134, 131209)

In [156]:
aisle_sim = pd.DataFrame(cosine_similarity(basket_mc),columns=basket_sets.columns,index=basket_sets.columns)

In [157]:
aisle_sim.fillna(0, inplace=True)

In [158]:
rules.to_csv('./data/aisle_rules.csv')

In [38]:
aisle_sim[aisle_sim.columns[1]].drop(aisle_sim.columns[1]).sort_values(ascending=False).head(3)

frozen juice           0.165058
beauty                 0.161477
baby bath body care    0.154951
Name: specialty cheeses, dtype: float64

In [39]:
def recommender(df,num_col):
    return df[df.columns[num_col]].drop(df.columns[num_col]).sort_values(ascending=False).head(3)

## Recommender

In [98]:
pd.DataFrame(recommender(aisle_sim,112)).T

,beauty,baby bath body care,baby accessories
frozen juice,0.680271,0.667037,0.666633


In [134]:
aisle_sim[aisle_sim.columns[1]].drop(aisle_sim.columns[1]).sort_values(ascending=False).head()

frozen juice                  0.165058
beauty                        0.161477
baby bath body care           0.154951
baby accessories              0.153701
specialty wines champagnes    0.148792
Name: specialty cheeses, dtype: float64

## Working with Products 

In [41]:
#test to  see # of aisles
print('# of unique product id =',len(opt['product_id'].unique()))
print('# of unique order id =',len(opt['order_id'].unique()))

# of unique product id = 39123
# of unique order id = 131209


## Market Basket for Products - Tasks
Need to filter out products that have been reordered to reduce the size of the product matrix. Once i have the reordered products, I can join the one hot encoded items to the order Id's


In [42]:
reorders = opt[opt['reordered'] == 1]

In [43]:
reorders['product_id'] = reorders['product_id'].astype('int64')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [44]:
reorders.head()
reorders.shape

(828824, 9)

In [45]:
reorders.dtypes

order_id              int64
product_id            int64
add_to_cart_order     int64
reordered             int64
product_name         object
aisle_id             object
department_id         int64
department           object
aisle                object
dtype: object

In [46]:
# get list of hi volume products (products that occurr mo' than 1 tiempo)
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [47]:
# mask the reorders dataframe to only incluse dem hi vol prods, yo
reorders = reorders[reorders['product_id'].isin(hivol)]

In [48]:
#reorders['hi_dem'] = 
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [49]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [50]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [52]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [53]:
products.shape

(13087, 4823)

In [54]:
basket = hidem_ord[['order_id']].join(products)

In [55]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [56]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [57]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  if __name__ == '__main__':


In [58]:
len(pro_dict)
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [59]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [60]:
basket_sets.sum().head()

Organic 4% Milk Fat Whole Milk Cottage Cheese    1
Lightly Smoked Sardines in Olive Oil             0
Spring Water                                     2
Cage Free Extra Large Grade AA Eggs              0
Roasted Turkey                                   0
dtype: int64

In [61]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [132]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules[['antecedants','consequents','confidence','lift']]

,antecedants,consequents,confidence,lift
0,(Orange Mango Chicken),"(Organic Nonfat Yogurt, Peach)",0.080000,27.301818
1,"(Organic Nonfat Yogurt, Peach)",(Orange Mango Chicken),0.181818,27.301818
2,(Cheese Shredded Mozzarella Reduced Fat 2%),(Russet Potato Bag),0.090909,37.919192
3,(Russet Potato Bag),(Cheese Shredded Mozzarella Reduced Fat 2%),0.222222,37.919192
4,(Light Sour Cream),(Beef Skirt Steak),0.076923,16.986425
5,(Beef Skirt Steak),(Light Sour Cream),0.176471,16.986425
6,(Beef Skirt Steak),(Cheese Party Pizza),0.117647,33.972851
7,(Cheese Party Pizza),(Beef Skirt Steak),0.153846,33.972851
8,"(Organic Nonfat Yogurt, Peach)",(Multi Grain Sandwich Thins),0.272727,24.971175
9,(Multi Grain Sandwich Thins),"(Organic Nonfat Yogurt, Peach)",0.073171,24.971175


## Market Basket for Products

In [131]:
rules[(rules['lift'] >= 100) & (rules['confidence']>.5)]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,"(Chocolate Almond Crunch Ice Cream, Medium Roa...",(Cheese Shredded Mozzarella Reduced Fat 2%),0.000799,0.005860,0.000533,0.666667,113.757576,0.000528,2.982419
44,"(Chocolate Almond Crunch Ice Cream, Duck Eggs)",(Orange Mango Chicken),0.000799,0.006660,0.000533,0.666667,100.106667,0.000527,2.980021
91,(Orchard Peach Sorbet),(Orange Mango Chicken),0.000533,0.006660,0.000533,1.000000,150.160000,0.000529,inf
93,"(White Beauty Bar Soap, Chocolate Almond Crunc...",(Orange Mango Chicken),0.000799,0.006660,0.000533,0.666667,100.106667,0.000527,2.980021
102,(Cauliettes Culinary Cuts),(Vanilla With Caramel Low Fat Ice Cream Cone),0.000533,0.004529,0.000533,1.000000,220.823529,0.000530,inf
106,(Puffs Simply White Cheddar Cheese Flavored Sn...,(Original Multigrain Crackers),0.000799,0.001865,0.000533,0.666667,357.523810,0.000531,2.994406
125,(Organic Cashew Carrot Ginger Soup),(Berry Punch Flavored Fruit Drink),0.000533,0.005860,0.000533,1.000000,170.636364,0.000530,inf
136,(Organic Pink Lady (Cripps) Apple),(Red Vine Tomato),0.000799,0.000533,0.000533,0.666667,1251.333333,0.000532,2.998402
137,(Red Vine Tomato),(Organic Pink Lady (Cripps) Apple),0.000533,0.000799,0.000533,1.000000,1251.333333,0.000532,inf
139,(Whipping Cream Ultra Pasteurized),(Vanilla With Caramel Low Fat Ice Cream Cone),0.000533,0.004529,0.000533,1.000000,220.823529,0.000530,inf


In [133]:
#Export data for tableau analysis.
rules.to_csv('./data/prod_rules.csv')

## Product Recommender

In [65]:
basket_mc = mean_center_rows(basket_sets)

In [66]:
product_sim = pd.DataFrame(cosine_similarity(basket_mc),columns=basket_sets.columns,index=basket_sets.columns)

In [129]:
pd.DataFrame(recommender(product_sim,3999)).T

,Rice Pilaf,Organic Freeze Dried Strawberries,Baby Persian Cucumbers
Organic Long Grain Brown Rice,1.0,1.0,1.0


In [101]:
#product_sim.to_csv('./data/prod_sim.csv')

In [130]:
product_sim

,Organic 4% Milk Fat Whole Milk Cottage Cheese,Lightly Smoked Sardines in Olive Oil,Spring Water,Cage Free Extra Large Grade AA Eggs,Roasted Turkey,Organic Raspberries,Organic Whole Strawberries,Natural Spring Water,Whole Milk Greek Blended Vanilla Bean Yogurt,Bag of Organic Bananas,...,Grated Parmesan Romano Cheese,Unscented One-Ply Bathroom Tissues,Diet Cherry Coke,Creamy Peanut Butter High Protein Bars,Green Tea,Beet Hummus,Blue Plus Bleach,Simply 100® Blueberry Blended Non-Fat Greek Yogurt,Simply 100 Peach Blended Non-Fat Greek Yogurt,ProActive Health Adult Chicken Dry Dog Food
Organic 4% Milk Fat Whole Milk Cottage Cheese,1.000000,0.022343,-0.000122,0.022343,0.022343,0.022343,0.022343,0.000035,0.000035,0.022343,...,0.022343,-0.000855,0.022343,-0.000855,-0.001210,-0.000172,-0.001590,-0.001626,-0.000172,-0.004775
Lightly Smoked Sardines in Olive Oil,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
Spring Water,-0.000122,0.000843,1.000000,0.000843,0.000843,0.000843,0.000843,-0.000415,-0.000415,0.000843,...,0.000843,-0.000916,0.000843,-0.000916,-0.001297,-0.000562,-0.001436,-0.001591,-0.000562,-0.003544
Cage Free Extra Large Grade AA Eggs,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
Roasted Turkey,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
Organic Raspberries,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
Organic Whole Strawberries,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
Natural Spring Water,0.000035,0.008245,-0.000415,0.008245,0.008245,0.008245,0.008245,1.000000,-0.000380,0.008245,...,0.008245,-0.001149,0.008245,-0.001149,-0.001626,-0.000587,-0.001884,-0.002043,-0.000587,-0.004934
Whole Milk Greek Blended Vanilla Bean Yogurt,0.000035,0.008245,-0.000415,0.008245,0.008245,0.008245,0.008245,-0.000380,1.000000,0.008245,...,0.008245,-0.001149,0.008245,-0.001149,-0.001626,-0.000587,-0.001884,-0.002043,-0.000587,-0.004934
Bag of Organic Bananas,0.022343,1.000000,0.000843,1.000000,1.000000,1.000000,1.000000,0.008245,0.008245,1.000000,...,1.000000,-0.024108,1.000000,-0.024108,-0.034118,0.001192,-0.049085,-0.048263,0.001192,-0.159772
